# Performance Considerations for Linear Algebra

## BLAS/LAPACK

After `using LinearAlgebra`, **Julia speaks linear algebra fluently**.

Performing linear algebra operations on a computer is, of course, an old problem. Lots of amazing libraries have been written - mostly in Fortran - which have been optimized over decades. Basically all high-level programming languages use these libraries, including R, Python, and Julia.

Linear algebra in Julia is largely implemented by calling [BLAS](http://www.netlib.org/blas/)/[LAPACK](http://www.netlib.org/lapack/) functions. As per default, Julia uses [OpenBLAS](https://github.com/xianyi/OpenBLAS).

* **BLAS**: a collection of low-level matrix and vector arithmetic operations ("multiply two matrices", "multiply a matrix by vector").
* **LAPACK**:  a collection of higher-level linear algebra operations. Things like matrix factorizations (LU, LLt, QR, SVD, Schur, etc) that are used to do things like “find the eigenvalues of a matrix”, or “find the singular values of a matrix”, or “solve a linear system”.

(Sparse matrices are more difficult and there exist different collections of routines, one of which is [SuiteSparse](http://faculty.cse.tamu.edu/davis/suitesparse.html).)

**Why do I have to care?**

* There are multiple BLAS/LAPACK implementations (e.g. by different vendors) and switching between them can lead to **big speedups**! 😄
* Since you might be leaving the world of Julia code, **you loose easy inspectability and type genericity**. 😢

### [Libblastrampoline](https://github.com/JuliaLinearAlgebra/libblastrampoline)

*BLAS and LAPACK demuxing library*

Allows one to readily **switch between BLAS/LAPACK implementations at runtime**. Practically, one uses small wrapper packages like

* [MKL.jl](https://github.com/JuliaLinearAlgebra/MKL.jl)
* [BLISBLAS.jl](https://github.com/carstenbauer/BLISBLAS.jl)

#### Example: OpenBLAS -> MKL with [MKL.jl](https://github.com/JuliaLinearAlgebra/MKL.jl)

In [ ]:
using LinearAlgebra

In [ ]:
BLAS.get_config()

In [ ]:
BLAS.set_num_threads(4)

In [ ]:
using BenchmarkTools
@btime A * B setup=(A = rand(100,100); B = rand(100,100););

In [ ]:
using MKL

In [ ]:
BLAS.get_config()

In [ ]:
BLAS.set_num_threads(4)

In [ ]:
@btime A * B setup=(A = rand(100,100); B = rand(100,100););

## Fast linear algebra with multiple dispatch

In [ ]:
N = 100
t = 1
μ = -0.5

H = diagm(0 => fill(μ, N), 1 => fill(-t, N-1), -1 => fill(-t, N-1))

In [ ]:
f(M, v) = v' * M * v

In [ ]:
v = rand(N);

In [ ]:
@btime f($H, $v);

In [ ]:
typeof(H)

As long as the code is generic (respects the informal `AbstractArray` interface), we can use the same piece of code for completely different array types. There are a number of [special matrix](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#Special-matrices) types are available out-of-the-box and [the ecosystem offers even more](https://github.com/JuliaMatrices).

Let's utilize the sparsity of `H` by indicating it through a type.

In [ ]:
using SparseArrays
Hsparse = sparse(H)

In [ ]:
typeof(Hsparse)

In [ ]:
@btime f($Hsparse, $v);

That's a solid speedup!

Our `H` isn't just sparse, but actually tridiagonal. Let's try to exploit that.

In [ ]:
Htri = Tridiagonal(H)

In [ ]:
@btime f($Htri, $v);

Choosing the best type (and therewith an algorithm) can be tricky and one has to play around a bit.

# Core messages of this Notebook

* **Trying a different BLAS/LAPACK** can often give speedups for free.
* **Indicate properties and structure of a matrix**, like hermiticity or sparsity, through types. Fallback to generic types only if you run into method errors.